# Sample 3: Cloud Deploy

Reference: cloud.google.com/deploy/docs/overview

## Deploy an app to GKE using Cloud Deploy

Reference: https://cloud.google.com/deploy/docs/deploy-app-gke

In [2]:
# gcloud init

In [5]:
!echo $(gcloud config get-value project)

training1027


In [4]:
# Make sure the default ompute Engine service account has sufficient permissions
# Add the clouddeploy.jobRunner role
!gcloud projects add-iam-policy-binding $(gcloud config get-value project) \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/clouddeploy.jobRunner"

Updated IAM policy for project [training1027].
bindings:
- members:
  - serviceAccount:service-487762169828@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/cloudbuild.builds.editor
- members:
  - serviceAccount:service-487762169828@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/clouddeploy.admin
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  role: roles/clouddeploy.jobRunner
- members:
  - serviceAccount:service-487762169828@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-487762169828@container-engine-robot.iam.gs

In [7]:
# Add the Kubernetes developer permissions
!gcloud projects add-iam-policy-binding $(gcloud config get-value project) \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/container.developer"

Updated IAM policy for project [training1027].
bindings:
- members:
  - serviceAccount:service-487762169828@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/cloudbuild.builds.editor
- members:
  - serviceAccount:service-487762169828@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:matt-training@training1027.iam.gserviceaccount.com
  role: roles/clouddeploy.admin
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  role: roles/clouddeploy.jobRunner
- members:
  - serviceAccount:service-487762169828@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com

In [8]:
# Add the iam.serviceAccountUser role, which includes the actAs permission to deploy to the runtime
!gcloud iam service-accounts add-iam-policy-binding $(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --member=serviceAccount:$(gcloud projects describe $(gcloud config get-value project) \
    --format="value(projectNumber)")-compute@developer.gserviceaccount.com \
    --role="roles/iam.serviceAccountUser" \
    --project=$(gcloud config get-value project)

Updated IAM policy for serviceAccount [487762169828-compute@developer.gserviceaccount.com].
bindings:
- members:
  - serviceAccount:487762169828-compute@developer.gserviceaccount.com
  - serviceAccount:487762169828@cloudbuild.gserviceaccount.com
  role: roles/iam.serviceAccountUser
etag: BwYIreLaEwQ=
version: 1


### Create Google Kubernetes Engine clusters

In [12]:
!gcloud container clusters create-auto quickstart-cluster-qsdev \
    --project=$(gcloud config get-value project) \
    --region=us-west1 && gcloud container clusters create-auto quickstart-cluster-qsprod \
    --project=$(gcloud config get-value project) \
    --region=us-west1

Note: The Pod address range limits the maximum size of the cluster. Please refer to https://cloud.google.com/kubernetes-engine/docs/how-to/flexible-pod-cidr to learn how to optimize IP address allocation.
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being confi
gured...⠼                                                                      
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being deplo
yed...⠼                                                                        
Creating cluster quickstart-cluster-qsdev in us-west1... Cluster is being healt
h-checked (master is healthy)...done.                                          
Created [https://container.googleapis.com/v1/projects/training1027/zones/us-west1/clusters/quickstart-cluster-qsdev].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-west1/quickstart-cluster-qsdev?project=training1027
CRITICAL: ACTION REQUIRED: gke-gc

### Prepare Skaffold configuration and Kubernetes manifest

In [13]:
!mkdir deploy-gke-quickstart

In [14]:
pwd

'/home/jupyter/CI-CD-GCP/sample-3-Cloud-Deploy'

In [15]:
cd deploy-gke-quickstart

/home/jupyter/CI-CD-GCP/sample-3-Cloud-Deploy/deploy-gke-quickstart


In [16]:
pwd

'/home/jupyter/CI-CD-GCP/sample-3-Cloud-Deploy/deploy-gke-quickstart'

In [17]:
%%writefile skaffold.yaml
apiVersion: skaffold/v2beta16
kind: Config
deploy:
  kubectl:
    manifests:
      - k8s-*

Writing skaffold.yaml


In [18]:
%%writefile k8s-pod.yaml
apiVersion: v1
kind: Pod
metadata:
  name: getting-started
spec:
  containers:
  - name: nginx
    image: my-app-image

Writing k8s-pod.yaml


### Create your delivery pipeline and targets

In [19]:
%%writefile clouddeploy.yaml
apiVersion: deploy.cloud.google.com/v1
kind: DeliveryPipeline
metadata:
  name: my-gke-demo-app-1
description: main application pipeline
serialPipeline:
  stages:
  - targetId: qsdev
    profiles: []
  - targetId: qsprod
    profiles: []
---

apiVersion: deploy.cloud.google.com/v1
kind: Target
metadata:
  name: qsdev
description: development cluster
gke:
  cluster: projects/training1027/locations/us-west1/clusters/quickstart-cluster-qsdev
---

apiVersion: deploy.cloud.google.com/v1
kind: Target
metadata:
  name: qsprod
description: production cluster
gke:
  cluster: projects/training1027/locations/us-west1/clusters/quickstart-cluster-qsprod

Writing clouddeploy.yaml


In [24]:
# Register your pipeline and targets with the Cloud Deploy service
!gcloud deploy apply --file=clouddeploy.yaml --region=us-west1 \
    --project=$(gcloud config get-value project)

Waiting for the operation on resource projects/training1027/locations/us-west1/
deliveryPipelines/my-gke-demo-app-1...done.                                    
Created Cloud Deploy resource: projects/training1027/locations/us-west1/deliveryPipelines/my-gke-demo-app-1.
Waiting for the operation on resource projects/training1027/locations/us-west1/
targets/qsdev...done.                                                          
Created Cloud Deploy resource: projects/training1027/locations/us-west1/targets/qsdev.
Waiting for the operation on resource projects/training1027/locations/us-west1/
targets/qsprod...done.                                                         
Created Cloud Deploy resource: projects/training1027/locations/us-west1/targets/qsprod.


### Create a release

In [26]:
!gcloud deploy releases create test-release-001 \
  --project=$(gcloud config get-value project) \
  --region=us-west1 \
  --delivery-pipeline=my-gke-demo-app-1 \
  --images=my-app-image=gcr.io/google-containers/nginx@sha256:f49a843c290594dcf4d193535d1f4ba8af7d56cea2cf79d1e9554f077f1e7aaa

Creating temporary tarball archive of 3 file(s) totalling 859 bytes before compression.
Uploading tarball of [.] to [gs://8aa5b76878204bd29a86a85d0500b451_clouddeploy/source/1698394112.49276-3cb5e373a9fc43018e0820346a77b9a0.tgz]
Waiting for operation [operation-1698394113885-608ae31d8c609-eedceb49-4104fabc]
...done.                                                                       
Created Cloud Deploy release test-release-001.
Creating rollout projects/training1027/locations/us-west1/deliveryPipelines/my-gke-demo-app-1/releases/test-release-001/rollouts/test-release-001-to-qsdev-0001 in target qsdev
Waiting for rollout creation operation to complete...done.                     


### Promote the release

### View the results in Google Cloud console

In [ ]:
# It didn't work!

### Clean up

In [ ]:
# gcloud container clusters delete quickstart-cluster-qsdev --region=us-west1 --project=$(gcloud config get-value project)
# gcloud container clusters delete quickstart-cluster-qsprod --region=us-west1 --project=$(gcloud config get-value project)
# gcloud deploy delete --file=clouddeploy.yaml --force --region=us-west1 --project=$(gcloud config get-value project)